In [4]:
from gensim.models import KeyedVectors
import pandas as pd

In [2]:
wv_from_text = KeyedVectors.load_word2vec_format("../src/toastynews.vec", binary=False)

In [3]:
wv = wv_from_text

In [32]:
wv.most_similar(["滑板", "踩單車"], ["單車"], topn=100)

[('跳傘', 0.544535219669342),
 ('溜冰', 0.5368380546569824),
 ('跑跳', 0.5199945569038391),
 ('滑翔', 0.4985131025314331),
 ('滑行', 0.49788039922714233),
 ('玩水', 0.4943167269229889),
 ('打波', 0.4911043047904968),
 ('滑水', 0.4895446002483368),
 ('跳上去', 0.4874389171600342),
 ('滑下', 0.47861266136169434),
 ('打籃球', 0.47582077980041504),
 ('滑雪板', 0.4752342402935028),
 ('腳踏車', 0.4749809503555298),
 ('踩車', 0.47264739871025085),
 ('彈跳', 0.4684738218784332),
 ('滑翔傘', 0.4674450159072876),
 ('踩下', 0.46541908383369446),
 ('跑邊', 0.4646003544330597),
 ('跳跳', 0.4639034569263458),
 ('飛邊', 0.4635488986968994),
 ('玩著', 0.4630483090877533),
 ('踏板', 0.46169763803482056),
 ('飛住', 0.45984572172164917),
 ('散步', 0.45917415618896484),
 ('打球', 0.4590715765953064),
 ('跑跑跳跳', 0.4588293731212616),
 ('游水', 0.4586943984031677),
 ('彈飛', 0.4573536515235901),
 ('衝浪', 0.4573422372341156),
 ('吹泡泡', 0.4567471742630005),
 ('攀爬', 0.4552578032016754),
 ('跳起', 0.45473960041999817),
 ('滑浪', 0.45473337173461914),
 ('跳街舞', 0.454491853713989

In [44]:
wv.most_similar("百葉窗")

[('百葉', 0.7031192779541016),
 ('大窗', 0.6824648976325989),
 ('窗門', 0.6610889434814453),
 ('小窗', 0.6562526822090149),
 ('門窗', 0.6499421000480652),
 ('窗', 0.6499367952346802),
 ('通風窗', 0.631084680557251),
 ('鋁窗', 0.630899965763092),
 ('窗戶', 0.6242531538009644),
 ('鐵窗', 0.6165266633033752)]

In [16]:
wv.similarity("垃圾", "堆") # good

0.3643237

In [ ]:
# Read excel file as a pandas DataFrame
df = pd.read_csv('HKSL_src.csv')

# Create a new column called "corr_cantonese"
df.insert(6,'1st','')
# df.insert(7,'mapped_NewFileName','')
df.insert(7,'3rd','')
df.insert(8,'5th','')
df.insert(9,'10th','')
df.insert(10,'-1','')

wordlist = df['cantonese_orig'].tolist()
filenamelist = df['NewFileName'].tolist()

# Loop through the Cantonese_Filtered column
for index, word in enumerate(wordlist):
    # Check if word is empty or not in vocab
    if not word or (word not in wv.key_to_index):
        df.at[index, '1st'] = 'NOTIN'
        df.at[index, 'cantonese_map_furthest'] = 'NOTIN'
    else:
        # Find the most similar word in the wordlist
        # most_similar_word = None
        # highest_similarity = -1
        # least_similar_word = None
        # lowest_similarity = 0
        similarityList = []
        similarityWordList = []
        
        for innerword in wordlist:
            if innerword == word: 
                continue
            if not innerword or (innerword not in wv.key_to_index): 
                continue
            similarity = wv.similarity(word, innerword)
            
            # if similarity > highest_similarity:
            #     most_similar_word = innerword
            #     highest_similarity = similarity
            # elif similarity < highest_similarity:
            #     least_similar_word = innerword
            #     lowest_similarity = similarity
                
            # method 2
            similarityList.append(similarity)
            similarityList.sort(reverse=True)
            idx = similarityList.index(similarity)
            similarityWordList.insert(idx, innerword)   
            
            f = open(f'words/{word}.txt', "w", encoding="utf-8")
            # f.write(most_similar_word)
            f.write(word)
            f.write("\n")
            for i in range(len(similarityList)):
                f.write(str(similarityWordList[i]))
                f.write(" ")
                f.write(str(similarityList[i]))
                if(i==0 or i==2 or i==4 or i==9 or i==len(similarityList)-1):
                    f.write(f" <-- {i+1}")
                f.write("\n")
            f.close()
            
                 
        # Fill in the corr_cantonese column with the most similar word found
        # df.at[index, '1st'] = most_similar_word
        # df.at[index, 'mapped_NewFileName'] = filenamelist[index]
        # df.at[index, 'cantonese_map_furthest'] = least_similar_word
        
        # method 2
        df.at[index, '1st'] = similarityWordList[0]
        # df.at[index, 'mapped_NewFileName'] = filenamelist[index]
        df.at[index, '3rd'] = similarityWordList[2]
        df.at[index, '5th'] = similarityWordList[4]
        df.at[index, '10th'] = similarityWordList[9]
        df.at[index, '-1'] = similarityWordList[-1]

# Save the DataFrame to a new excel file
df.to_excel('HKSL_added.xlsx', index=False)
